<a href="https://colab.research.google.com/github/yf591/rag_chatbot_demo/blob/main/rag_chatbot_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# rag-chatbot-demoのデモ
このノートブックでは以下の内容を理解＆学習しながら実行することができます。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ

準備として、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。

### 注意事項
「rag-chatbot-demo」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 or L4 or A100」を選択してください。（推奨はL4以上）

### ⚠️ Google Colab特有の注意点

**FAISS問題の対処**
- Google Colabでは`faiss-gpu`が利用できないため、`faiss-cpu`を使用します。そのため若干精度が落ちます。
- このノートブックでは自動的に適切なバージョンをインストールします

**よくあるエラーと対処法**
- 「関連する情報が見つかりませんでした」→ ベクトルストアの問題（後述のトラブルシューティングセルを実行）
- GPU認識しない → ランタイム再起動
- メモリ不足 → 高RAMオプション有効化

# 環境変数の設定


GitHubから演習用のコードをCloneします。

In [ ]:
!git clone https://github.com/yf591/rag_chatbot_demo.git

必要なAPIトークンを.envに設定します。

「rag-chatbot-demo」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /content/rag_chatbot_demo
load_dotenv(find_dotenv())

# rag-chatbot-demoの起動


ディレクトリ「rag-chatbot-demo」に移動します。

In [ ]:
%cd /content/rag_chatbot_demo

必要なライブラリをインストールします。

## ⚠️ 重要：ベクトルストアの確認

ここでは**ベクトルストアを作成**して、FAISSが正しく動作するかを確認します。

まず、次のセルを実行して
- FAISSライブラリの動作確認
- ベクトルストアファイルの存在確認
- GPU環境の確認

を行います。

In [ ]:
# Google Colab用の依存関係インストール

# 1. まずFAISSを個別にインストール（Google Colab対応）
print("=== FAISSライブラリのインストール ===")
!pip install faiss-cpu  # Google ColabではCPU版を使用

# 2. その他の依存関係をインストール
print("\n=== その他の依存関係インストール ===")
!pip install streamlit
!pip install transformers>=4.36.0
!pip install torch>=2.0.0
!pip install langchain>=0.3.0
!pip install langchain-community>=0.3.0
!pip install langchain-huggingface>=0.1.0
!pip install sentence-transformers>=2.2.0
!pip install PyMuPDF
!pip install accelerate>=0.20.0
!pip install bitsandbytes>=0.41.0
!pip install huggingface_hub>=0.16.0
!pip install python-dotenv
!pip install pyngrok
!pip install toml

print("\n✅ 依存関係のインストール完了")

In [ ]:
# Google Colab用ベクトルストア確認・作成
print("=== ベクトルストア状態確認 ===\n")

# まず軽量版でベクトルストア作成（Google Colab推奨）
print("🔄 Google Colab用軽量ベクトルストア作成中...")
!python pdf_processor_light.py

print("\n=== インストール確認 ===")

try:
    import faiss
    print(f"✅ FAISS: バージョン {faiss.__version__}")
except ImportError as e:
    print(f"❌ FAISS インポートエラー: {e}")

try:
    from langchain_community.vectorstores import FAISS as LangChainFAISS
    from langchain_huggingface import HuggingFaceEmbeddings
    print("✅ LangChain FAISS統合: OK")
except ImportError as e:
    print(f"❌ LangChain インポートエラー: {e}")

# ベクトルストアの存在確認
import os
if os.path.exists('./vector_store/index.faiss'):
    print("✅ ベクトルストアファイル: 存在")

    # ファイルサイズ確認
    faiss_size = os.path.getsize('./vector_store/index.faiss')
    pkl_size = os.path.getsize('./vector_store/documents.pkl') if os.path.exists('./vector_store/documents.pkl') else 0
    print(f"📊 index.faiss: {faiss_size/1024:.1f}KB")
    print(f"📊 documents.pkl: {pkl_size/1024:.1f}KB")
else:
    print("❌ ベクトルストアファイル: 見つかりません")
    print("💡 PDF前処理が必要な可能性があります")

print("\n=== GPU確認 ===")
import torch
print(f"CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

print("\n=== ベクトルストア動作テスト ===")
try:
    from langchain_huggingface import HuggingFaceEmbeddings
    from langchain_community.vectorstores import FAISS

    # 埋め込みモデル初期化
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    # ベクトルストア読み込みテスト
    vector_store = FAISS.load_local(
        "./vector_store",
        embeddings,
        allow_dangerous_deserialization=True
    )

    # 簡単な検索テスト
    test_results = vector_store.similarity_search("福山", k=1)
    if test_results:
        print(f"✅ ベクトルストア動作: 正常（{len(test_results)}件取得）")
        print(f"📄 テスト結果: {test_results[0].page_content[:100]}...")
    else:
        print("⚠️ ベクトルストア動作: 検索結果なし")

except Exception as e:
    print(f"❌ ベクトルストアエラー: {e}")
    print("🔧 修復が必要です - 次のトラブルシューティングセルを実行してください")

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

## 🔧 トラブルシューティング（必要時のみ実行）

**もし「関連する情報が見つかりませんでした」というエラーが出る場合**は、以下のセルを実行してベクトルストアを修復してください。

In [ ]:
# ベクトルストアの修復（エラー時のみ実行）
print("=== ベクトルストア修復処理 ===")

try:
    # ベクトルストアの状態確認
    !python check_vector_store.py

    print("\n=== ベクトルストア修復実行 ===")
    !python fix_vector_store.py

    print("\n=== 修復後の確認 ===")
    !python check_vector_store.py

except Exception as e:
    print(f"エラー: {e}")
    print("\n💡 手動でPDF前処理を実行します...")
    !python pdf_processor_light.py

In [ ]:
# RAGシステムの動作テスト
print("=== RAGシステム動作テスト ===")

try:
    from rag_system_simple import RAGSystemSimple

    # シンプル版でテスト
    rag_simple = RAGSystemSimple()

    # テスト質問
    test_question = "福山市について教えてください"
    print(f"テスト質問: {test_question}")

    # 文書検索テスト
    docs = rag_simple.retrieve_documents(test_question, k=2)

    if docs:
        print(f"\n✅ 検索成功: {len(docs)}件の関連文書を発見")
        for i, doc in enumerate(docs[:1]):  # 最初の1件を表示
            print(f"\n--- 文書 {i+1} ---")
            print(f"ソース: {doc.get('metadata', {}).get('source', '不明')}")
            print(f"内容: {doc.get('content', '')[:200]}...")
        print("\n🎉 RAGシステムは正常に動作しています！")
    else:
        print("\n❌ 文書検索に失敗しました")
        print("ベクトルストアに問題がある可能性があります")

except Exception as e:
    print(f"\n❌ テストエラー: {e}")
    print("上記の修復処理を実行してください")

### ✅ 成功の確認方法

上記のテストで以下が表示されれば準備完了です。

1. **FAISS**: ✅ バージョン表示
2. **LangChain FAISS統合**: ✅ OK
3. **ベクトルストアファイル**: ✅ 存在
4. **検索成功**: ✅ X件の関連文書を発見
5. **RAGシステムは正常に動作しています！**

❌ エラーが出る場合は、上記のトラブルシューティングセルを実行してください。

## Streamlitアプリの起動

stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [ ]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

rag-chatbot-demoでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()